<a href="https://colab.research.google.com/github/qxu6/WBSE/blob/main/daily_progression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [ ]:
!pip install geemap

In [ ]:
import google.colab
import pandas as pd
from IPython.display import clear_output
import geemap
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import ee
import pytz
from pytz import timezone
import random
import sys
import math
from collections import defaultdict
from pprint import pprint
sys.setrecursionlimit(100000)

# Initialize timezone variables
gmt = timezone('GMT')
pst = timezone('US/Pacific')

# Initialize Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=PuVRViE-BwS3aXbDGpKpsX3VQZDJCHcnDgIPl346iek&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiVuudmJuJoQuyDa8g3qFi5M6WUZTaKIQRI3iwKC5U-UI4rSvpPV0w

Successfully saved authorization token.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#INPUTS

In [ ]:
# Input fire perimeters ee.FeatureCollection
mtbs_fc = ee.FeatureCollection('projects/workgroup4/assets/mtbs/mtbs_1984_2020')
# fraps_fc = ee.FeatureCollection('projects/workgroup4/assets/fraps_1984_2020')
fraps_fc = ee.FeatureCollection('projects/workgroup4/assets/fraps_ver20211206')

fire_fc_name = 'fraps'

# Input MODIS fire detections
# modis_fc = ee.FeatureCollection('projects/workgroup4/assets/mtbs/firms_m_2000_2020')
modis_fc = ee.FeatureCollection('projects/workgroup4/assets/CA_M_2019_2021')

# Input VIIRS fire detections
# viirs_fc = ee.FeatureCollection('projects/workgroup4/assets/mtbs/firms_sv_2000_2020')
viirs_fc = ee.FeatureCollection('projects/workgroup4/assets/CA_SV_2019_2021')

# Input J1 fire detections
# j1_fc = ee.FeatureCollection('projects/workgroup4/assets/mtbs/firms_j1v_2000_2020')
j1_fc = ee.FeatureCollection('projects/workgroup4/assets/CA_J1V_2019_2021')

# Combine feature collections
combined_fc = modis_fc.merge(viirs_fc)
combined_fc = combined_fc.merge(j1_fc)

# Get fire years from ee.FeatureCollection
# fire_years = ee.List(fire_fc.aggregate_array('Year')).getInfo()

# Get fire years from range of years
# fire_years = list(range(2002,2006,1))
# fire_years.sort(reverse=True)
# fire_years = [str(x) for x in fire_years]

# Get specific fire years
# fire_years = ['2020']

# Set as 'year' or 'e_id'
process_type = 'year'
process_list = ['2020']

# process_type = 'e_id'
# process_list = ['2020_175','2020_226','2020_234','2020_236','2020_237','2020_6','2020_300','2020_312','2020_317','2020_319','2020_331']

# Set polygon buffer size in m for detection point search polygon
buff_f = 750

# Set point buffer size in m
buff_pt = 500

# Set raster pixel size in m
r_size = 500

# Set export pixel size in m
export_size = 500

# Set the date offsets for start and end date in months
inc_start = 0
inc_end = 3

# Set time zone
origin = gmt
local = pst

asset_path = 'WG4/Daily Progression/Assets/'
drive_path = '/content/drive/MyDrive/'+asset_path
ee_path = 'projects/workgroup4/assets/daily_progression/'

# FUNCTIONS

In [ ]:
def get_fc_df(df):
  """Convert fire boundary feature collection information to Pandas dataframe.

  Args:
      df (ee.FeatureCollection): Fire boundaries as an Earth Engine Feature Collection

  Returns:
      pd.DataFrame: Pandas dataframe of fire boundary information
  """
  # Get list of column names from feature collection
  df_prop = ee.List(df.first().propertyNames()).getInfo()

  # Initialize iteration storage
  col_data = {}

  # Initialize size list for error checking
  col_size = []

  # Iterate for every column name
  for col_name in df_prop:
    # Aggregate property array into list
    col = ee.List(df.aggregate_array(col_name)).getInfo()
a
    # Store list in iteration storage
    col_data[col_name] = col

    # Store size for error checking
    col_size.append(len(col))

  # Create error dictionary for filtering
  col_size = pd.DataFrame({'name':df_prop,'size':col_size})

  # Filter indicies that do not have the same length as the max index
  col_err = col_size[col_size['size']<col_size['size'].max()]['name']
  
  # Delete columns with size errors
  for name in col_err:
    del col_data[name]

  # Return cleaned dictionary as pd.DataFrame
  pd_df = pd.DataFrame(col_data)

  return pd_df

In [ ]:
# def fraps_to_dt(df):
#   # Change date from ms to s
#   df['Date'] = df['Date']/1000

#   # Remove dates before 1984
#   df = df[df['Date']>0]

#   # Change date to local time
#   df['datetime'] = pd.to_datetime(df['Date'], unit='s').dt.tz_localize(local)

#   return df

def fraps_to_dt(df):
  df['datetime'] = pd.to_datetime(df['Date_str'], format='%m/%d/%Y').dt.tz_localize(local)

  return df

def mtbs_to_dt(df):
  # Change date to local time
  df['datetime'] = pd.to_datetime(df['Date']).dt.tz_localize(local)

  return df

def process_dates(df):
  df['year'] = df['datetime'].dt.strftime('%Y')

  df['s_date'] = df['datetime'].apply(lambda x: x + relativedelta(months = inc_start))

  df['e_date'] = df['datetime'].apply(lambda x: x + relativedelta(months = inc_end))

  return df

def process_name(df):
  df['f_name'] = df['Incid_Name'].apply(lambda x: clean_text(x))
  df['e_id'] = df['UCM_ID'].apply(lambda x: clean_text(x))
  return df

def subset_fire(df,k,v):
  df = df[df[k]==v].copy()
  return df

def get_fire_df(fc):
  if fc == 'fraps':
    df = get_fc_df(fraps_fc)
    df = fraps_to_dt(df)

  elif fc == 'mtbs':
    df = get_fc_df(mtbs_fc)
    df = mtbs_to_dt(df)

  else:
    print('***ERROR: NOT A VALID INPUT***')

  df = process_dates(df)
  df = process_name(df)

  return df

def set_fire_fc(fc):
  if fc == 'fraps':
    fc = fraps_fc

  elif fc == 'mtbs':
    fc = mtbs_fc

  else:
    print('***ERROR: NOT A VALID INPUT***')

  return fc

def get_feature(f_fc,fc_id,df_id):

  # Create fire feature
  fire = ee.Feature(f_fc.filter(ee.Filter.Or(ee.Filter.eq(fc_id,df_id))).first())

  return fire

def get_bounds(fire,buff):
  # Create fire bounds for export
  fire_bounds = fire.geometry().buffer(buff)
  fire_bounds = fire_bounds.bounds()

  return fire_bounds

def filter_points(fire,pt_fc,buff):
  """Filter fire detection points within fire boundary.

  Args:
      fire (ee.Feature): Fire boundary as an Earth Engine Feature
      pt_fc (ee.FeatureCollection): Fire detection points as an Earth Engine Feature Collection
      buff (int): Fire boundary buffer distance in meters

  Returns:
      ee.FeatureCollection: Subset of fire detection points within fire boundary
  """
  # Get fire boundary and buffer
  fire = fire.buffer(buff)

  # Filter point collection
  poi = pt_fc.filterBounds(fire.geometry().getInfo())

  return poi

def split_4(bounds):
  x_coords = []
  y_coords = []
  for i in list(range(0,4)):
    x_coord = bounds['coordinates'][0][i][0]
    y_coord = bounds['coordinates'][0][i][1]

    x_coords.append(x_coord)
    y_coords.append(y_coord)

  x_coords = list(set(x_coords))
  y_coords = list(set(y_coords))

  min_x = min(x_coords)
  max_x = max(x_coords)
  min_y = min(y_coords)
  max_y = max(y_coords)

  len_x = abs(max_x-min_x)/3
  len_y = abs(max_y-min_y)/3

  x_1 = min_x
  x_2 = min_x+(len_x)
  x_3 = max_x-(len_x)
  x_4 = max_x

  y_1 = min_y
  y_2 = min_y+(len_y)
  y_3 = max_y-(len_y)
  y_4 = max_y

  b_1 = [x_1,y_1,x_2,y_2]
  b_2 = [x_2,y_1,x_3,y_2]
  b_3 = [x_3,y_1,x_4,y_2]
  b_4 = [x_1,y_2,x_2,y_3]
  b_5 = [x_2,y_2,x_3,y_3]
  b_6 = [x_3,y_2,x_4,y_3]
  b_7 = [x_1,y_3,x_2,y_4]
  b_8 = [x_2,y_3,x_3,y_4]
  b_9 = [x_3,y_3,x_4,y_4]

  split_bounds = [b_1,b_2,b_3,b_4,b_5,b_6,b_7,b_8,b_9]
  return split_bounds

def get_pt_df(df):
  """Convert fire boundary feature collection information to Pandas dataframe.

  Args:
      df (ee.FeatureCollection): Fire boundaries as an Earth Engine Feature Collection

  Returns:
      pd.DataFrame: Pandas dataframe of fire boundary information
  """
  # Get list of column names from feature collection
  df_prop = ee.List(df.first().propertyNames()).getInfo()

  # Initialize iteration storage
  col_data = {}

  # Initialize size list for error checking
  col_size = []

  # Iterate for every column name
  for col_name in df_prop:
    # Aggregate property array into list
    col = ee.List(df.aggregate_array(col_name)).getInfo()

    # Store list in iteration storage
    col_data[col_name] = col

    # Store size for error checking
    col_size.append(len(col))

  # Create error dictionary for filtering
  col_size = pd.DataFrame({'name':df_prop,'size':col_size})

  # Filter indicies that do not have the same length as the max index
  col_err = col_size[col_size['size']<col_size['size'].max()]['name']

  # Delete columns with size errors
  for name in col_err:
    del col_data[name]

  # Return cleaned dictionary as pd.DataFrame
  pd_df = pd.DataFrame(col_data)

  return pd_df

def clean_confidence(df):
  """Clean fire detection data point confidence labels.
      'l' -> 0 Low confidence goes from 0 to 30. Label stored as 0 for filtering.
      'n' -> 55 Nominal confidence goes from 30 to 80. Label stored as 55 for filtering.
      'h' -> 100 High confidence goes from 80 to 100. Label stored as 100 for filtering.
  Args:
      df (pd.DataFrame): Pandas dataframe of fire detection points.

  Returns:
      pd.DataFrame: Pandas dataframe of fire detection points with cleaned confidence labels.
  """

  # Copy CONFIDENCE column to integer relabeled column.
  df['conf_int'] = df['CONFIDENCE']

  # Low confidence ranges from 0 to 30. Store low confidence as 0 for filtering
  df.loc[df['conf_int']=='l',['conf_int']] = 0

  # Nominal confidence ranges from 30 to 80. Store nominal confidence as the average 55.
  df.loc[df['conf_int']=='n',['conf_int']] = 55

  # High confidence ranges from 80 to 100. Store high confidence as 100.
  df.loc[df['conf_int']=='h',['conf_int']] = 100

  # Change column datatype to int.
  df = df.astype({'conf_int':'int32'})
  
  return df

def pop_dupe_coord(df):
  """Pop duplicate coordinates from point collection dataframe

  Args:
      df (pd.DataFrame): Pandas dataframe of points. Necessary columns:
                            LONGITUDE: x-coordinate of the point
                            LATITUDE: y-coordinate of the point

  Returns:
      pd.DataFrame: Pandas dataframe of points with duplicated coordinates
  """

  # Sort dataframe by acquisition date to align duplicates
  df.sort_values('ACQ_DATE')

  # Select points in the dataframe that have identical latitudes and longitudes
  df = df[df.duplicated(subset=['LATITUDE','LONGITUDE'])].copy()

  return df

def remove_dupe_coords(df):
  """Remove duplicate coordinates from point collection dataframe. Use in conjunction with pop_dupe_coords to preserve duplicated coordinates.

  Args:
      df (pd.DataFrame): Pandas dataframe of points. Necessary columns:
                            LONGITUDE: x-coordinate of the point
                            LATITUDE: y-coordinate of the point

  Returns:
      pd.DataFrame: Pandas dataframe of points with duplicate coordinates removed
  """

  # Sort dataframe by acquisition date to align duplicates
  # Remove duplicates and preserve the earliest instance of the point
  df = df.sort_values('ACQ_DATE').drop_duplicates(subset=['LATITUDE','LONGITUDE'],keep='first')

  return df

def create_dt(row):
  """Create the datetime object and date string for each fire detection point.

  Args:
      row (row): Row in Pandas Data Frame that includes following columns:
                  ACQ_DATE: Date of detection
                  ACQ_TIME: Time of detection
                  datetime: Empty column to store datetime
                  date_string: Empty column to store date string

  Returns:
      row: Row in Pandas Data Frame with updated datetime and date_string columns
  """
  # Get Year, Month, and Day from ACQ_DATE
  date = datetime.fromtimestamp(row['ACQ_DATE']/1000)

  # Get Hours and Minutes from ACQ_TIME
  hours = int(row['ACQ_TIME'][0:2])
  minutes = int(row['ACQ_TIME'][2:4])

  # Process datetime info into datetime object and convert from GMT to PST
  dt = datetime(date.year,date.month,date.day,hours,minutes,tzinfo = origin).astimezone(local)
  row['datetime']  = dt
  row['date_string'] = dt.strftime('%Y-%m-%d %H:%M')
  return row

def get_tt(date):
  """Convert datetime to 366 time tuple. Stores the hours and minutes as a fraction of a 24-hour day.

  Args:
      date (datetime64): Date stored as a datetime object

  Returns:
      float64: Time tuple aaa.bbbb where aaa is the day stored as a 366 day-of-year and bbbb is a 4-digit fraction of a 24-hour day.
  """

  # Store the datetime.timetuple() day object
  days = date.timetuple()[-2]

  # Convert the datetime.timetuple() hours object to fraction of day
  hours = (date.timetuple()[3] + 1) / 24

  # Convert the datetime.timetuple() minutes object to fraction of day 
  minutes = (date.timetuple()[4] + 1) / 1440

  # Combine time tuple objects and preserve to 4 decimal places
  tt = round(days+hours+minutes,5)

  return tt

def get_point_df(points,fire,min_date='None'):
  combined_df = get_fc_df(points)
  combined_df = clean_confidence(combined_df)
  combined_dupes = pop_dupe_coord(combined_df)
  combined_df = remove_dupe_coords(combined_df)

  low_conf = combined_df[combined_df['conf_int'] < 30].copy()
  combined_df = combined_df[combined_df['conf_int'] >= 30]
  
  # Create time tuples and date delta for points
  # Initialize datetime and date_string columns for create_dt function
  combined_df['datetime'] = ''
  combined_df['date_string'] = ''

  # Run create_dt function
  combined_df = combined_df.apply(create_dt,axis=1)

  # Filter the dataframe for points that happen within the fire date
  len_before_dates = len(combined_df)

  combined_df = combined_df[(combined_df['datetime']>=fire['s_date'])&(combined_df['datetime']<fire['e_date'])]

  if (len(combined_df)==0) and (len_before_dates>0):
    print('All dates outside of time window.')
  else:
    pass

  # Initialize the minimum date for date delta calculation
  if min_date is 'None':
    min_date = combined_df['datetime'].min()
  else:
    pass

  # Run date delta calculation
  combined_df['date_d'] = combined_df['datetime'] - min_date

  # Store date delta in human readable format
  combined_df['date_d_ts'] = combined_df['date_d'].apply(lambda x: x.total_seconds())
  combined_df['date_d_days'] = combined_df['date_d_ts']/86400

  # Convert datetime to time tuple
  combined_df['date_tt'] = combined_df['datetime'].apply(get_tt)

  df_exports = {'dupes':combined_dupes,'low_conf':low_conf,'df':combined_df}
  return df_exports

def fc_to_raster(fc,reducer,prop,pt_scale,r_scale):
  """Convert Earth Engine feature collection of points to raster.

  Args:
      fc (ee.FeatureCollection): Earth Engine feature collection of points
      reducer (ee.Reducer): Earth Engine reduction method
      prop (string): Name of feature property to be rasterized
      pt_scale (int): Distance in meters to buffer points
      r_scale (int): Distance in meters to scale pixels

  Returns:
      [type]: [description]
  """
  # Buffer points to remove boundary cases in rasterization
  fc = fc.map(lambda x: x.buffer(pt_scale))

  # Reduce points to rasters and project to native Earth Engine projection with the given pixel scale.
  raster = fc.reduceToImage([prop],reducer).reproject('epsg:4326', None, r_scale)

  # Ensure raster values are stored as Float values
  raster = raster.toFloat()

  return raster

def df_to_fc(df,x,y,p_name,col):
  """Convert Pandas dataframe of points to Earth Engine Feature Collection

  Args:
      df (pd.DataFrame): Pandas dataframe of point information
      x (string): Name of Longitude column
      y (string): Name of Latitude column
      p_name (string): Name of Earth Engine feature property
      col (string): Name of feature property column

  Returns:
      ee.FeatureCollection: Earth Engine feature collection of points with property values
  """
  # Initialize empty list for storage
  features = []

  # Iterate over every row of the dataframe
  for index,row in df.iterrows():
    # Get the x,y coordinates for each point
    coords = [row[x],row[y]]

    # Create an ee.Geometry feature for the point
    geom = ee.Geometry.Point(coords)

    # Convert the ee.Geometry to an ee.Feature object
    feature = ee.Feature(geom)

    # Set the feature property
    feature = feature.set(p_name,row[col])

    # Add the ee.Feature object to the storage list
    features.append(feature)
  
  # Convert the storage list to an ee.FeatureCollection
  fc = ee.FeatureCollection(features)

  return fc

def pt_to_fc(combined_df):
  # Create point features with appended time tuples
  gen_tt = df_to_fc(combined_df,'LONGITUDE','LATITUDE','date','date_tt')

  # Create point features with appended date delta
  gen_date_d = df_to_fc(combined_df,'LONGITUDE','LATITUDE','date','date_d_days')

  fc_dict = {'tt':gen_tt,'dd':gen_date_d}
  return fc_dict

def create_rasters(fc_dict):
  agg_dict = {
      'min':ee.Reducer.min(),
      'mean':ee.Reducer.mean(),
      'max':ee.Reducer.max(),
      'count':ee.Reducer.count()
      }

  r_img_dict = {}
  for dt,fc in fc_dict.items():
    for agg,reducer in agg_dict.items():
      r_key = dt+'_'+agg
      r_value = fc_to_raster(fc,reducer,'date',buff_pt,r_size)
      r_img_dict[r_key]= r_value
      
  return r_img_dict

def reduce_img_coll(proc,ic):
  if proc == 'min':
    img = ic.reduce(ee.Reducer.min())

  elif proc == 'mean':
    img = ic.reduce(ee.Reducer.mean())

  elif proc == 'max':
    img = ic.reduce(ee.Reducer.max())

  elif proc == 'count':
    img = ic.reduce(ee.Reducer.sum())

  else:
    print('ERROR INVALID PROCESS TYPE AT IMAGE COLLECTION AGGREGATION')

  return img

def create_raster_img_dict(combined_df):
  date_type = 'date_d'
  date_list = list(set(combined_df[date_type]))
  full_dict = {}

  for date in date_list:
    date_df = combined_df[combined_df[date_type] == date]
    date_name = str(date).replace('.','_')
    proc_name = 'proc_'+str(date_name)

    fc_dict = pt_to_fc(date_df)
    r_img_dict = create_rasters(fc_dict)
    full_dict[proc_name] = r_img_dict

  img_list_dict = {}
  for key,value in full_dict.items():
    for stat in list(full_dict[key].keys()):
      if stat in img_list_dict.keys():
        img_list_dict[stat].append(full_dict[key][stat])
      else:
        img_list_dict[stat]=[full_dict[key][stat]]

  img_dict = {}
  for key,value in img_list_dict.items():
    ic = ee.ImageCollection(img_list_dict[key])
    stat = key.split('_')[1]
    img_dict[key] = reduce_img_coll(stat,ic)
  return img_dict

def create_subset_img_dict(fire,combined_df,fire_bounds,combined_fc):
  subset_img_dict = {}

  num_detect = len(combined_df)
  subset_img_dict['num_points'] = num_detect
  subset_img_dict['num_dates'] = len(list(set(combined_df['date_d'])))
  date_d_min = combined_df['datetime'].min()

  if num_detect < 25000:
    print('Num detections < 25000. Creating image dictionary.')
    fc_dict = pt_to_fc(combined_df)
    
    print('Writing to subset_img_dict.')
    subset_img_dict['img_dict'] = create_raster_img_dict(combined_df)
    subset_img_dict['bounds'] = fire_bounds
    subset_img_dict['points'] = combined_df
    subset_img_dict['large_fire'] = False

  else:
    print('Num detections > 25000. Creating split image dictionary.')
    split_bounds = split_4(fire_bounds.getInfo())
    i_split = 0

    list_raster_img_dict = []
    list_raster_bounds_dict = []
    list_raster_points_dict = []

    for split in split_bounds:
      fire_bounds = ee.Geometry.Rectangle(split)
      fire_feature = ee.Feature(fire_bounds).buffer(2*buff_f)

      i_split+=1

      points = filter_points(fire_feature,combined_fc,750)
      point_dict = get_point_df(points,fire,date_d_min)
      combined_df = point_dict['df']
      fc_dict = pt_to_fc(combined_df)
      raster_img_dict = create_raster_img_dict(combined_df)

      list_raster_img_dict.append(raster_img_dict)
      list_raster_bounds_dict.append(fire_bounds)
      list_raster_points_dict.append(combined_df)

    print('Writing to subset_img_dict.')
    subset_img_dict['img_dict'] = list_raster_img_dict
    subset_img_dict['bounds'] = list_raster_bounds_dict
    subset_img_dict['points'] = list_raster_points_dict
    subset_img_dict['large_fire'] = True

  return subset_img_dict

def export_fire_image(img,asset_path,ee_path,name,pixel_size,region):
  """Export Earth Engine image to Google Drive

  Args:
      img (ee.Image): Earth Engine image
      asset_path (string): Google Drive folder where the export will be stored
      name (string): Export file name
      pixel_size (int): Size of the export pixels
      region (ee.Geometry): Boundary of the export image
  """
  # Store image kwargs as dictionary
  d_img_kwarg = {
    'image': img,
    'folder': asset_path,
    'description': name,
    'scale':pixel_size,
    'region': region,
    'maxPixels': int(1e13)
  }

  # Create image export task
  d_img_export = ee.batch.Export.image.toDrive(**d_img_kwarg)
  
  e_img_kwarg = {
    'image': img,
    'description': name,
    'assetId': ee_path,
    'scale':pixel_size, 
    'region': region, 
    'maxPixels': int(1e13)
  }
  
  e_img_export = ee.batch.Export.image.toAsset(**e_img_kwarg)
  # Start image export
  d_img_export.start()
  e_img_export.start()
  return

def clean_symbols(text):
  for ch in ['#','.',',','&','.','"',"'",',','\r','\n','?']:
        if ch in text:
            text = text.replace(ch,'')
  return text

def clean_spaces(text):
  for ch in ['(',')',' ','+','/']:
        if ch in text:
            text = text.replace(ch,'_')
  return text

def clean_text(text):
  text = clean_symbols(text)
  text = clean_spaces(text)
  return text      

# RUN

In [ ]:
# Initialize Fire DF (PROCESS BOTTLENECK)
print('Creating fire polygon dataframe.')

fire_df = get_fire_df(fire_fc_name)
fire_df = fire_df.sort_values(by='Area', ascending=False)
fire_df = fire_df[450:]
fire_fc = set_fire_fc(fire_fc_name)

# Iterate over process list
print('Begin iteration {} process list.'.format(process_type))

for p_item in process_list:
  # Subset to process subset
  print('Creating {} subset for {}.'.format(process_type,p_item))

  subset_df = subset_fire(fire_df,process_type,p_item)

  # Set image export folder
  img_folder = asset_path+'images/'+p_item+'/'
  csv_folder = drive_path+'csvs/'+p_item+'/'

  # Get the number of fires for the iterator
  num_fires = len(subset_df)

  # Initialize the fire iterator
  i_fire = 0

  # Create empty columns for appended data
  subset_df[['num_points','num_dates','status','error','img_dict','large_fire','fire_bounds']] = ''

  for index,row in subset_df.iterrows():
    i_fire +=1

    print('Working on {} fire {} of {}.'.format(row['f_name'],i_fire,num_fires))

    # Get fire feature
    fire = get_feature(fire_fc,'UCM_ID',row['UCM_ID'])

    # Create fire bounds for export
    fire_bounds = get_bounds(fire,buff_f)

    # Get detection points from EE (PROCESS BOTTLENECK)

    print('Getting detection points.')
    points = filter_points(fire,combined_fc,750)

    # Try to create point df


    if points.size().getInfo()>0:
      point_dict = get_point_df(points,row)

      try:
        combined_df = point_dict['df']

      except Exception as e:
        print('************Error at {} {}. {}.Could not build point features.'.format(row['f_name'],row['e_id'],e))
        subset_df.loc[index,'status'] = 'ERROR'
        subset_df.loc[index,'error'] = e
        subset_img_dict['points'].to_csv(csv_folder+pt_csv_path)

      if len(combined_df)>0:
        try:
          print('Creating subset image dictionary.')
          subset_img_dict = create_subset_img_dict(row,combined_df,fire_bounds,combined_fc)

          print('Subset image dictionary created. Appending information to Subset DataFrame.')
          subset_df.loc[index,'num_points'] = subset_img_dict['num_points']
          subset_df.loc[index,'num_dates'] = subset_img_dict['num_dates']
          subset_df.loc[index,'large_fire'] = subset_img_dict['large_fire']

          if subset_img_dict['large_fire']:
            print('====={} is a Large Fire. Running split export.====='.format(row['f_name']))
            i_large_dict = 0

            while i_large_dict < len(subset_img_dict['img_dict']):
              img_dict = subset_img_dict['img_dict'][i_large_dict]
              fire_bounds = subset_img_dict['bounds'][i_large_dict]
              # Point DF Export
              pt_csv_name = row['f_name']+'_points_'+str(i_large_dict)+'_'+str(i_large_dict)+'_MOSAIC.csv'
              pt_csv_path = row['e_id']+'_'+row['year']+'_'+pt_csv_name

              for k in list(img_dict.keys()):
                img = img_dict[k]
                incid_name = row['f_name']+'_'+k+'_'+str(i_large_dict)+'_'+str(i_large_dict)+'_MOSAIC'
                export_name = row['e_id']+'_'+row['year']+'_'+incid_name
                ee_asset = ee_path+export_name
                export_fire_image(img,img_folder,ee_asset,export_name,export_size,fire_bounds)

              i_large_dict+=1

          else:
            print('=====Exporting {}.====='.format(row['f_name']))
            img_dict = subset_img_dict['img_dict']
            # Point DF Export
            pt_csv_name = row['f_name']+'_points.csv'
            pt_csv_path = row['e_id']+'_'+row['year']+'_'+pt_csv_name
            subset_img_dict['points'].to_csv(csv_folder+pt_csv_path)

            for k in list(img_dict.keys()):
              img = img_dict[k]
              incid_name = row['f_name']+'_'+k
              export_name = row['e_id']+'_'+row['year']+'_'+incid_name
              ee_asset = ee_path+export_name
              export_fire_image(img,img_folder,ee_asset,export_name,export_size,fire_bounds)

        except Exception as e:
          print('************Error at {} {}. {}. Could not build point features.'.format(row['f_name'],row['e_id'],e))
          subset_df.loc[index,'status'] = 'ERROR'
          subset_df.loc[index,'error'] = e

      elif (len(combined_df) == 0) and (len(point_dict['low_conf']) > 0):
        subset_df.loc[index,'status'] = 'ERROR'
        subset_df.loc[index,'error'] = 'Only low confidence points.'
        print('************Error at {}. Only low confidence points.'.format(row['f_name']))  

      else:
        subset_df.loc[index,'status'] = 'ERROR'
        subset_df.loc[index,'error'] = 'Not enough points.'
        print('************Error at {}. Not enough points.'.format(row['f_name']))

    else:
      subset_df.loc[index,'status'] = 'ERROR'
      subset_df.loc[index,'error'] = 'Not enough points.'
      print('************Error at {}. Not enough points.'.format(row['f_name']))

    subset_df.loc[index,'status'] = 'Generated'
  
    #Export process list
    process_csv_name = 'process_list_'+p_item+'.csv'
    subset_df.to_csv(csv_folder+process_csv_name)

In [ ]:
print(fire_df.Date)

# OLD VERSION DO NOT RUN

##Inputs


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Input fire perimeters ee.FeatureCollection
mtbs_fc = ee.FeatureCollection('projects/workgroup4/assets/mtbs/mtbs_1984_2020')
fraps_fc = ee.FeatureCollection('projects/workgroup4/assets/fraps_1984_2020')

fire_fc = fraps_fc

# Input MODIS fire detections
modis_fc = ee.FeatureCollection('projects/workgroup4/assets/mtbs/firms_m_2000_2020')

# Input VIIRS fire detections
viirs_fc = ee.FeatureCollection('projects/workgroup4/assets/mtbs/firms_sv_2000_2020')

# Input J1 fire detections
j1_fc = ee.FeatureCollection('projects/workgroup4/assets/mtbs/firms_j1v_2000_2020')

# Combine feature collections
combined_fc = modis_fc.merge(viirs_fc)
combined_fc = combined_fc.merge(j1_fc)

# Get fire years from ee.FeatureCollection
# fire_years = ee.List(fire_fc.aggregate_array('Year')).getInfo()

# Get fire years from range of years
# fire_years = list(range(2002,2006,1))
# fire_years.sort(reverse=True)
# fire_years = [str(x) for x in fire_years]

# Get specific fire years
fire_years = ['2020']

# Select pixel output size
pixel_size = 500

# Set maximum date range in months
max_date = 3

# Set time zone
origin = gmt
local = pst

# Asset Path
asset_path = 'WG4/Daily Progression/Assets/'
drive_path = '/content/drive/MyDrive/'+asset_path

##Functions

###Earth Engine Functions

In [ ]:
def filter_points(fire,pt_fc,buff):
  """Filter fire detection points within fire boundary.

  Args:
      fire (ee.Feature): Fire boundary as an Earth Engine Feature
      pt_fc (ee.FeatureCollection): Fire detection points as an Earth Engine Feature Collection
      buff (int): Fire boundary buffer distance in meters

  Returns:
      ee.FeatureCollection: Subset of fire detection points within fire boundary
  """
  # Get fire boundary and buffer
  fire = fire.buffer(buff)

  # Filter point collection
  poi = pt_fc.filterBounds(fire.geometry().getInfo())

  return poi

def buffer_pts(geom):
  """Buffer geometries in an Earth Engine feature collection. Distance must be initialized in function to use with Earth Engine.

  Args:
      geom (ee.FeatureCollection): Earth Engine feature collection of geometries

  Returns:
      ee.FeatureCollection: Buffered Earth Engine feature collection of geometries
  """
  return geom.buffer(1000)

def fc_to_raster(fc,reducer,prop,pt_scale,r_scale):
  """Convert Earth Engine feature collection of points to raster.

  Args:
      fc (ee.FeatureCollection): Earth Engine feature collection of points
      reducer (ee.Reducer): Earth Engine reduction method
      prop (string): Name of feature property to be rasterized
      pt_scale (int): Distance in meters to buffer points
      r_scale (int): Distance in meters to scale pixels

  Returns:
      [type]: [description]
  """
  # Buffer points to remove boundary cases in rasterization
  fc = fc.map(lambda x: x.buffer(pt_scale))

  # Reduce points to rasters and project to native Earth Engine projection with the given pixel scale.
  raster = fc.reduceToImage([prop],reducer).unmask(-9999).reproject('epsg:4326', None, r_scale)

  # Ensure raster values are stored as Float values
  raster = raster.toFloat()

  return raster

def export_fire_image(img,asset_path,name,pixel_size,region):
  """Export Earth Engine image to Google Drive

  Args:
      img (ee.Image): Earth Engine image
      asset_path (string): Google Drive folder where the export will be stored
      name (string): Export file name
      pixel_size (int): Size of the export pixels
      region (ee.Geometry): Boundary of the export image
  """
  # Store image kwargs as dictionary
  img_kwarg = {
    'image': img,
    'folder': asset_path,
    'description': name,
    'scale':pixel_size,
    'region': region,
    'maxPixels': int(1e13)
  }

  # Create image export task
  img_export = ee.batch.Export.image.toDrive(**img_kwarg)

  # Start image export
  img_export.start()
  return

###Table Conversion Functions

In [ ]:
def get_fc_df(df):
  """Convert fire boundary feature collection information to Pandas dataframe.

  Args:
      df (ee.FeatureCollection): Fire boundaries as an Earth Engine Feature Collection

  Returns:
      pd.DataFrame: Pandas dataframe of fire boundary information
  """
  # Get list of column names from feature collection
  df_prop = ee.List(df.first().propertyNames()).getInfo()

  # Initialize iteration storage
  col_data = {}

  # Initialize size list for error checking
  col_size = []

  # Iterate for every column name
  for col_name in df_prop:
    # Aggregate property array into list
    col = ee.List(df.aggregate_array(col_name)).getInfo()

    # Store list in iteration storage
    col_data[col_name] = col

    # Store size for error checking
    col_size.append(len(col))

  # Create error dictionary for filtering
  col_size = pd.DataFrame({'name':df_prop,'size':col_size})

  # Filter indicies that do not have the same length as the max index
  col_err = col_size[col_size['size']<col_size['size'].max()]['name']

  # Delete columns with size errors
  for name in col_err:
    del col_data[name]c

  # Return cleaned dictionary as pd.DataFrame
  pd_df = pd.DataFrame(col_data)

  return pd_df

def df_to_fc(df,x,y,p_name,col):
  """Convert Pandas dataframe of points to Earth Engine Feature Collection

  Args:
      df (pd.DataFrame): Pandas dataframe of point information
      x (string): Name of Longitude column
      y (string): Name of Latitude column
      p_name (string): Name of Earth Engine feature property
      col (string): Name of feature property column

  Returns:
      ee.FeatureCollection: Earth Engine feature collection of points with property values
  """
  # Initialize empty list for storage
  features = []

  # Iterate over every row of the dataframe
  for index,row in df.iterrows():
    # Get the x,y coordinates for each point
    coords = [row[x],row[y]]

    # Create an ee.Geometry feature for the point
    geom = ee.Geometry.Point(coords)

    # Convert the ee.Geometry to an ee.Feature object
    feature = ee.Feature(geom)

    # Set the feature property
    feature = feature.set(p_name,row[col])

    # Add the ee.Feature object to the storage list
    features.append(feature)
  
  # Convert the storage list to an ee.FeatureCollection
  fc = ee.FeatureCollection(features)

  return fc


###Python Functions

In [ ]:
def clean_confidence(df):
  """Clean fire detection data point confidence labels.
      'l' -> 0 Low confidence goes from 0 to 30. Label stored as 0 for filtering.
      'n' -> 55 Nominal confidence goes from 30 to 80. Label stored as 55 for filtering.
      'h' -> 100 High confidence goes from 80 to 100. Label stored as 100 for filtering.
  Args:
      df (pd.DataFrame): Pandas dataframe of fire detection points.

  Returns:
      pd.DataFrame: Pandas dataframe of fire detection points with cleaned confidence labels.
  """

  # Copy CONFIDENCE column to integer relabeled column.
  df['conf_int'] = df['CONFIDENCE']

  # Low confidence ranges from 0 to 30. Store low confidence as 0 for filtering
  df.loc[df['conf_int']=='l',['conf_int']] = 0

  # Nominal confidence ranges from 30 to 80. Store nominal confidence as the average 55.
  df.loc[df['conf_int']=='n',['conf_int']] = 55

  # High confidence ranges from 80 to 100. Store high confidence as 100.
  df.loc[df['conf_int']=='h',['conf_int']] = 100

  # Change column datatype to int.
  df = df.astype({'conf_int':'int32'})
  
  return df

def pop_dupe_coord(df):
  """Pop duplicate coordinates from point collection dataframe

  Args:
      df (pd.DataFrame): Pandas dataframe of points. Necessary columns:
                            LONGITUDE: x-coordinate of the point
                            LATITUDE: y-coordinate of the point

  Returns:
      pd.DataFrame: Pandas dataframe of points with duplicated coordinates
  """

  # Sort dataframe by acquisition date to align duplicates
  df.sort_values('ACQ_DATE')

  # Select points in the dataframe that have identical latitudes and longitudes
  df = df[df.duplicated(subset=['LATITUDE','LONGITUDE'])].copy()

  return df

def remove_dupe_coords(df):
  """Remove duplicate coordinates from point collection dataframe. Use in conjunction with pop_dupe_coords to preserve duplicated coordinates.

  Args:
      df (pd.DataFrame): Pandas dataframe of points. Necessary columns:
                            LONGITUDE: x-coordinate of the point
                            LATITUDE: y-coordinate of the point

  Returns:
      pd.DataFrame: Pandas dataframe of points with duplicate coordinates removed
  """

  # Sort dataframe by acquisition date to align duplicates
  # Remove duplicates and preserve the earliest instance of the point
  df = df.sort_values('ACQ_DATE').drop_duplicates(subset=['LATITUDE','LONGITUDE'],keep='first')

  return df

def get_tt(date):
  """Convert datetime to 366 time tuple. Stores the hours and minutes as a fraction of a 24-hour day.

  Args:
      date (datetime64): Date stored as a datetime object

  Returns:
      float64: Time tuple aaa.bbbb where aaa is the day stored as a 366 day-of-year and bbbb is a 4-digit fraction of a 24-hour day.
  """

  # Store the datetime.timetuple() day object
  days = date.timetuple()[-2]

  # Convert the datetime.timetuple() hours object to fraction of day
  hours = (date.timetuple()[3] + 1) / 24

  # Convert the datetime.timetuple() minutes object to fraction of day 
  minutes = (date.timetuple()[4] + 1) / 1440

  # Combine time tuple objects and preserve to 4 decimal places
  tt = round(days+hours+minutes,5)

  return tt

def get_date_delta(date_d):
  """Convert datedelta object to day delta tuple.

  Args:
      date_d (datedelta): Date delta object

  Returns:
      float64: Date delta aaa.bbbb where aaa is the day stored as time since fire start and bbbb is a 4-digit fraction of a 24-hour day.
  """

  # Store the datedelta day object
  days = date_d.days

  # Convert the datedelta seconds object to fraction of day
  seconds = date_d.seconds / 86400

  # Combine date delta objects and preserve to 4 decimal places
  date_d = round(days+seconds,8)

  return date_d  

def get_strftime(date):
  """Convert date to YYYY-mm-dd string.

  Args:
      date (datetime): Datetime object

  Returns:
      string: Date string as YYYY-mm-dd
  """

  return date.strftime('%Y-%m-%d')

def get_datetime(date):
  """Convert YYYY-mm-dd date string to datetime object

  Args:
      date (string): Date string as YYYY-mm-dd

  Returns:
      datetime: Datetime object
  """

  return date.strptime('%Y-%m-%d')

def create_dt(row):
  """Create the datetime object and date string for each fire detection point.

  Args:
      row (row): Row in Pandas Data Frame that includes following columns:
                  ACQ_DATE: Date of detection
                  ACQ_TIME: Time of detection
                  datetime: Empty column to store datetime
                  date_string: Empty column to store date string

  Returns:
      row: Row in Pandas Data Frame with updated datetime and date_string columns
  """
  # Get Year, Month, and Day from ACQ_DATE
  date = datetime.fromtimestamp(row['ACQ_DATE']/1000)

  # Get Hours and Minutes from ACQ_TIME
  hours = int(row['ACQ_TIME'][0:2])
  minutes = int(row['ACQ_TIME'][2:4])

  # Process datetime info into datetime object and convert from GMT to PST
  dt = datetime(date.year,date.month,date.day,hours,minutes,tzinfo = origin).astimezone(local)
  row['datetime']  = dt
  row['date_string'] = dt.strftime('%Y-%m-%d %H:%M')
  return row

##Run

In [ ]:
# Create Fire Feature Data Frame
print('Begin Fire Features to Data Frame.')

# Run Feature Collection to Data Frame Function
fire_df = get_fc_df(fire_fc)
fire_df = fire_df.sort_values(by='Area', ascending=False)
fire_df = fire_df[0:50]

# Set date to current date
fire_df['Date'] = fire_df['Date'].astype(float)/1000 # RUN FOR FRAPS
fire_df = fire_df[fire_df['Date']>0]
fire_df['datetime'] = pd.to_datetime(fire_df['Date'], unit='s').dt.tz_localize(local) # RUN FOR FRAPS

#fire_df['datetime'] = pd.to_datetime(fire_df['Date']).dt.tz_localize(local) #RUN FOR MTBS

# Extract year from date time
fire_df['Year'] = fire_df['datetime'].dt.strftime('%Y')

# Get start date for every fire. Buffer months using integer values to extend start time.
fire_df['s_date'] = fire_df['datetime'].apply(lambda x:x + relativedelta(months = 0))

# Get end date for every fire. Default to 3 months after fire start to capture majority of fire activity.
fire_df['e_date'] = fire_df['datetime'].apply(lambda x:x + relativedelta(months = max_date))

# Clean Incident Names and assign it to f_name
fire_df['f_name'] = fire_df['Incid_Name'].apply(lambda x:x.replace(' ','_'))
fire_df['f_name'] = fire_df['Incid_Name'].apply(lambda x:x.replace('/','_'))

# Clean Event ID and assign it to e_id
fire_df['e_id'] = fire_df['UCM_ID'].apply(lambda x:x.replace(' ','_'))
fire_df['e_id'] = fire_df['UCM_ID'].apply(lambda x:x.replace('/','_'))

print('Dataframe built. Begin yearly processing.')

# Begin iteration for years
for focus_year in fire_years:

  print('Begin processing {} fires.'.format(focus_year))

  # Subset the dataframe to the focus year
  subset_fire_df = fire_df[fire_df['Year']==focus_year].copy()
  
  # Set image export folder to the focus year
  image_folder = asset_path+focus_year+'/'
  asset_folder = drive_path+focus_year+'/'

  # Get the number of fires for the iterator
  num_fires = len(subset_fire_df)

  # Initialize fire iterator
  i_fire = 0

  # Create empty columns for appended data
  subset_fire_df['num_points'] = ''
  subset_fire_df['num_dates'] = ''
  subset_fire_df['status'] = ''
  subset_fire_df['error'] = ''

  # Iterate for every fire in subset
  for index,row in subset_fire_df.iterrows():

    i_fire+=1

    print('Working on {} fire {} of {}.'.format(row['f_name'],i_fire,num_fires))

    try:
      # Create fire feature
      fire = ee.Feature(fire_fc.filter(ee.Filter.Or(ee.Filter.eq('UCM_ID',row['UCM_ID']))).first())

      # Create fire bounds for export
      fire_bounds = fire.geometry().bounds()
      fire_bounds = fire_bounds.buffer(750)

      # Create fire points for raster generation
      points = filter_points(fire,combined_fc,750)

      try:
        print('Creating fire dataframe for {} {}.'.format(row['f_name'],row['e_id']))
        # Convert point features to dataframe for analysis
        combined_df = get_fc_df(points)

        # Clean confidence values for points
        combined_df = clean_confidence(combined_df)

        # Remove duplicate points
        combined_dupes = pop_dupe_coord(combined_df)
        combined_df = remove_dupe_coords(combined_df)

        # Remove points with low confidence conf_int < 30.
        low_conf = combined_df[combined_df['conf_int'] < 30].copy()
        combined_df = combined_df[combined_df['conf_int'] >= 30]

        # Create time tuples and date delta for points
        # Initialize datetime and date_string columns for create_dt function
        combined_df['datetime'] = ''
        combined_df['date_string'] = ''

        # Run create_dt function
        combined_df = combined_df.apply(create_dt,axis=1)

        # Filter the dataframe for points that happen within the fire date
        combined_df = combined_df[(combined_df['datetime']>=row['s_date'])&(combined_df['datetime']<row['e_date'])]

        # Initialize the minimum date for date delta calculation
        min_date = combined_df['datetime'].min()

        # Run date delta calculation
        combined_df['date_d'] = combined_df['datetime'] - min_date

        # Store date delta in human readable format
        combined_df['date_d_ts'] = combined_df['date_d'].apply(lambda x: x.total_seconds())
        combined_df['date_d_days'] = combined_df['date_d_ts']/86400

        # Convert datetime to time tuple
        combined_df['date_tt'] = combined_df['datetime'].apply(get_tt)

      except Exception as e:
        print('************Error at {} {}. {}.Could not build point features.'.format(row['f_name'],row['e_id'],e))
        combined_df = pd.DataFrame()
        subset_fire_df.loc[index,'status'] = 'ERROR'
        subset_fire_df.loc[index,'error'] = e

      if len(combined_df)>0:
        # Only run the fire analysis if there is more than 1 point in the point collection
        print('Fire dataframe built. Processing raster.')

        # Create point features with appended time tuples
        gen_tt = df_to_fc(combined_df,'LONGITUDE','LATITUDE','date','date_tt')

        # Create point features with appended date delta
        gen_date_d = df_to_fc(combined_df,'LONGITUDE','LATITUDE','date','date_d_days')

        print('Generating {} {} Time Tuple Raster.'.format(row['f_name'],row['e_id']))

        # Assign the number of points to the full fire list
        subset_fire_df.loc[index,'num_points'] = len(combined_df)

        # Assign the number of unique dates to the full fire list
        subset_fire_df.loc[index,'num_dates'] = len(combined_df['datetime'].drop_duplicates())

        # Generate the time tuple raster
        raster_tt_min = fc_to_raster(gen_tt,ee.Reducer.min(),'date',500,500)
        raster_tt_mean = fc_to_raster(gen_tt,ee.Reducer.mean(),'date',500,500)
        raster_tt_max = fc_to_raster(gen_tt,ee.Reducer.max(),'date',500,500)

        print('Generating {} {} Date Delta Raster.'.format(row['f_name'],row['e_id']))

        # Generate the date delta raster
        raster_date_d_min = fc_to_raster(gen_date_d,ee.Reducer.min(),'date',500,500)
        raster_date_d_mean = fc_to_raster(gen_date_d,ee.Reducer.mean(),'date',500,500)
        raster_date_d_max = fc_to_raster(gen_date_d,ee.Reducer.max(),'date',500,500)

        print('Exporting {} {} Rasters.'.format(row['f_name'],row['e_id']))

        # Export raster to Google Drive
        export_fire_image(raster_tt_min,image_folder,'tt_min_'+row['e_id'],pixel_size,fire_bounds)
        export_fire_image(raster_tt_max,image_folder,'tt_max_'+row['e_id'],pixel_size,fire_bounds)
        export_fire_image(raster_tt_mean,image_folder,'tt_mean_'+row['e_id'],pixel_size,fire_bounds)

        export_fire_image(raster_date_d_min,image_folder,'dd_min_'+row['e_id'],pixel_size,fire_bounds)
        export_fire_image(raster_date_d_max,image_folder,'dd_max_'+row['e_id'],pixel_size,fire_bounds)
        export_fire_image(raster_date_d_mean,image_folder,'dd_mean_'+row['e_id'],pixel_size,fire_bounds)

        # Update fire status in full fire list
        subset_fire_df.loc[index,'status'] = 'Generated'

        # Export point CSV to Google Drive
        print('Exporting {} {} CSV.'.format(row['f_name'],row['e_id']))
        # combined_df.to_csv(asset_folder+row['e_id']+'_pts.csv')

      else:
        subset_fire_df.loc[index,'status'] = 'ERROR'
        subset_fire_df.loc[index,'error'] = 'Not enough points.'
        print('************Error at {}. Not enough points.'.format(row['f_name']))

    except Exception as e:
      print('Error at {} {}. {}.'.format(row['f_name'],row['e_id'],e))
      subset_fire_df.loc[index,'status'] = 'ERROR'
      subset_fire_df.loc[index,'error'] = e

    # Export full fire list for the year
    export_csv = subset_fire_df[['Date','Year','Incid_Name','f_name','e_id','s_date','e_date','num_points','num_dates','error']]
    export_csv.to_csv(asset_folder+'Image_List_'+focus_year+'.csv')

  print('{} fires processed.'.format(focus_year))

Begin Fire Features to Data Frame.


KeyError: ignored

##Vizualizer


In [ ]:
Map = geemap.Map()
Map.addLayer(raster_0.clip(fire_bounds).mask(raster_0.gt(0)),{'min':test_min,'max':test_max,'palette':["0080FF","FFFF00","FF8000","FF0000"]},'raster_0')
Map.addLayer(gen_fc)
Map

In [ ]:
Map = geemap.Map()
Map.addLayer(fire_bounds)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# Notes
- Remove clumps https://developers.google.com/earth-engine/guides/image_objects\
- Export coordinates CSV \
- Export duplicated coordinates CSV to "Duplicated" folder \

In [ ]:
def get_shape_dict(fc):
    """Returns the Earth Engine Feature Collection Metadata as a Pandas 
    dataframe.

    Args:
        fc (ee.FeatureCollection): ee.FeatureCollection of fire polygons. Name 
        and year should be stored as FIRE_NA and YEAR_, respectively.

    Returns:
        pd.DataFrame: Pandas dataframe of the fire names and years from the 
        Earth Engine Feature Collection.
    """
    fire_names = ee.List(fc.aggregate_array('FIRE_NA')).getInfo()
    year = ee.List(fc.aggregate_array('YEAR_')).getInfo()
    df = pd.DataFrame({'fire_name':fire_names,'year':year})
    return df

##Single Fire Exporter

In [ ]:
id_list = ['CA3924012311020180727']

In [ ]:
# Create Fire Feature Data Frame
print('Begin Fire Features to Data Frame.')

# Run Feature Collection to Data Frame Function
fire_df = get_fc_df(fire_fc)

# Set date to current date
fire_df['datetime'] = pd.to_datetime(fire_df['Date']).dt.tz_localize(time_zone)

# Extract year from date time
fire_df['Year'] = fire_df['datetime'].dt.strftime('%Y')

# Get start date for every fire. Buffer months using integer values to extend start time.
fire_df['s_date'] = fire_df['datetime'].apply(lambda x:x + relativedelta(months = 0))

# Get end date for every fire. Default to 3 months after fire start to capture majority of fire activity.
fire_df['e_date'] = fire_df['datetime'].apply(lambda x:x + relativedelta(months = max_date))

# Clean Incident Names and assign it to f_name
fire_df['f_name'] = fire_df['Incid_Name'].apply(lambda x:x.replace(' ','_'))
fire_df['f_name'] = fire_df['Incid_Name'].apply(lambda x:x.replace('/','_'))

# Clean Event ID and assign it to e_id
fire_df['e_id'] = fire_df['UCM_ID'].apply(lambda x:x.replace(' ','_'))
fire_df['e_id'] = fire_df['UCM_ID'].apply(lambda x:x.replace('/','_'))

print('Dataframe built. Begin yearly processing.')

# Begin iteration for fire list
for id in id_list:
  print('Begin processing specific fires.')

  # Subset the dataframe to the focus year
  subset_fire_df = fire_df[fire_df['e_id']==id].copy()
  
  # Set image export folder to the focus year
  image_folder = asset_path+'individual_fires/'

  # Get the number of fires for the iterator
  num_fires = len(subset_fire_df)

  # Initialize fire iterator
  i_fire = 0

  # Create empty columns for appended data
  subset_fire_df['num_points'] = ''
  subset_fire_df['num_dates'] = ''
  subset_fire_df['status'] = ''
  subset_fire_df['error'] = ''

  # Iterate for every fire in subset
  for index,row in subset_fire_df.iterrows():

    i_fire+=1

    print('Working on {} fire {} of {}.'.format(row['f_name'],i_fire,num_fires))

    try:
      # Create fire feature
      fire = ee.Feature(fire_fc.filter(ee.Filter.Or(ee.Filter.eq('UCM_ID',row['UCM_ID']))).first())

      # Create fire bounds for export
      fire_bounds = fire.geometry().bounds()
      fire_bounds = fire_bounds.buffer(750)

      # Create fire points for raster generation
      points = filter_points(fire,combined_fc,750)

      try:
        print('Creating fire dataframe for {} {}.'.format(row['f_name'],row['e_id']))
        # Convert point features to dataframe for analysis
        combined_df = get_fc_df(points)

        # Clean confidence values for points
        combined_df = clean_confidence(combined_df)

        # Remove duplicate points
        combined_dupes = pop_dupe_coord(combined_df)
        combined_df = remove_dupe_coords(combined_df)

        # Remove points with low confidence conf_int < 30.
        low_conf = combined_df[combined_df['conf_int'] < 30].copy()
        combined_df = combined_df[combined_df['conf_int'] >= 30]

        # Create time tuples and date delta for points
        # Initialize datetime and date_string columns for create_dt function
        combined_df['datetime'] = ''
        combined_df['date_string'] = ''

        # Run create_dt function
        combined_df = combined_df.apply(create_dt,axis=1)

        # Filter the dataframe for points that happen within the fire date
        combined_df = combined_df[(combined_df['datetime']>=row['s_date'])&(combined_df['datetime']<row['e_date'])]

        # Initialize the minimum date for date delta calculation
        min_date = combined_df['datetime'].min()

        # Run date delta calculation
        combined_df['date_d'] = combined_df['datetime'] - min_date

        # Store date delta in human readable format
        combined_df['date_d_ts'] = combined_df['date_d'].apply(lambda x: x.total_seconds())
        combined_df['date_d_days'] = combined_df['date_d_ts']/86400

        # Convert datetime to time tuple
        combined_df['date_tt'] = combined_df['datetime'].apply(get_tt)
      except Exception as e:
        print('************Error at {} {}. {}.Could not build point features.'.format(row['f_name'],row['e_id'],e))
        combined_df = pd.DataFrame()
        subset_fire_df.loc[index,'status'] = 'ERROR'
        subset_fire_df.loc[index,'error'] = e

      if len(combined_df)>0:
        # Only run the fire analysis if there is more than 1 point in the point collection
        print('Fire dataframe built. Processing raster.')

        # Create point features with appended time tuples
        gen_tt = df_to_fc(combined_df,'LONGITUDE','LATITUDE','date','date_tt')

        # Create point features with appended date delta
        gen_date_d = df_to_fc(combined_df,'LONGITUDE','LATITUDE','date','date_d_days')

        print('Generating {} {} Time Tuple Raster.'.format(row['f_name'],row['e_id']))

        # Assign the number of points to the full fire list
        subset_fire_df.loc[index,'num_points'] = len(combined_df)

        # Assign the number of unique dates to the full fire list
        subset_fire_df.loc[index,'num_dates'] = len(combined_df['datetime'].drop_duplicates())

        # Generate the time tuple raster
        raster_tt = fc_to_raster(gen_tt,ee.Reducer.mean(),'date',500,500)

        print('Generating {} {} Date Delta Raster.'.format(row['f_name'],row['e_id']))

        # Generate the date delta raster
        raster_date_d = fc_to_raster(gen_date_d,ee.Reducer.mean(),'date',500,500)

        # Combine the time tuple and date delta raster for export
        raster = raster_tt.addBands(raster_date_d)

        print('Exporting {} {} Raster.'.format(row['f_name'],row['e_id']))

        # Export raster to Google Drive
        # export_fire_image(raster,image_folder,row['e_id'],pixel_size,fire_bounds)
        # export_fire_image(raster_tt,image_folder,row['e_id']+'_tt',pixel_size,fire_bounds)
        # export_fire_image(raster_date_d,image_folder,row['e_id']+'_date_d',pixel_size,fire_bounds)

        # Update fire status in full fire list
        subset_fire_df.loc[index,'status'] = 'Generated'

        # Export point CSV to Google Drive
        # combined_df.to_csv(asset_folder+row['e_id']+'_pts.csv')

      else:
        subset_fire_df.loc[index,'status'] = 'ERROR'
        subset_fire_df.loc[index,'error'] = 'Not enough points.'
        print('************Error at {}. Not enough points.'.format(row['f_name']))

    except Exception as e:
      print('Error at {} {}. {}.'.format(row['f_name'],row['e_id'],e))
      subset_fire_df.loc[index,'status'] = 'ERROR'
      subset_fire_df.loc[index,'error'] = e

  # Export full fire list for the year
  export_csv = subset_fire_df[['Date','Year','Incid_Name','f_name','s_date','e_date','num_points','num_dates','error']]
  # export_csv.to_csv(asset_folder+'Image_List.csv')

  print('Fires processed.')

Begin Fire Features to Data Frame.
Dataframe built. Begin yearly processing.
Begin processing specific fires.
Working on RANCH fire 1 of 1.
Creating fire dataframe for RANCH CA3924012311020180727.
Fire dataframe built. Processing raster.
Generating RANCH CA3924012311020180727 Time Tuple Raster.
Error at RANCH CA3924012311020180727. 'DATE'.
Fires processed.


In [ ]:
len(combined_df['datetime'].drop_duplicates())

330

In [ ]:
min_tt = combined_df.date_tt.min()
max_tt = combined_df.date_tt.max()
min_date_d = combined_df.date_d_days.min()
max_date_d = combined_df.date_d_days.max()

In [ ]:
Map = geemap.Map()
Map.addLayer(raster_tt.clip(fire_bounds).mask(raster_tt.gt(0)),{'min':min_tt,'max':max_tt,'palette':["0080FF","FFFF00","FF8000","FF0000"]},'raster_0')
Map

In [ ]:
Map = geemap.Map()
Map.addLayer(raster_date_d.clip(fire_bounds).mask(raster_date_d.gte(0)),{'min':min_date_d,'max':max_date_d,'palette':["0080FF","FFFF00","FF8000","FF0000"]},'raster_0')
Map